In [1]:
# Import package
import pandas as pd
import numpy as np
import os
import sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from src.db.vector_store import VectorStore
from src.rag.embedding_base import EmbeddingBase
from src.rag.mistral_embedding import MistralEmbedding
from src.rag.gemini_embedding import GoogleEmbedding
from src.rag.rag_retriever import RagRetriever, RAGMetrics

In [2]:
# Define the API keys here
MISTRAL_API_KEY= ""
GEMINI_API_KEY = ""

In [3]:
# Modele embedding mistral
google_embedding = GoogleEmbedding(GEMINI_API_KEY)

In [4]:
vector_store = VectorStore(
    gemini_api_key=GEMINI_API_KEY,
    mistral_api_key=MISTRAL_API_KEY  # Paramètre obligatoire mais non utilisé ici
)

In [5]:
retriever = RagRetriever(
    modele_embedding=google_embedding,
    vector_store=vector_store,
    nombre_resultats=5,
    prix_tokens={"input": 0.50, "output": 1.50}  # Exemple de coûts
)

In [6]:
# Ajout de documents
documents = [
    """Les Transformers sont des architectures de deep learning introduites en 2017.
    Ils utilisent des mécanismes d'attention et sont particulièrement efficaces pour le NLP.""",
    
    """Le RAG (Retrieval-Augmented Generation) combine recherche d'information et génération de texte.
    Il utilise typiquement un système de retrieval vectoriel et un LLM pour produire des réponses."""
]

try:
    document_ids = retriever.ajouter_documents(
        textes=documents,
        metadonnees=[
            {"source": "wikipedia", "auteur": "IA"},
            {"source": "arxiv", "annee": 2020}
        ]
    )
    print(f"Documents ajoutés avec IDs : {document_ids}")
except Exception as e:
    print(f"Erreur lors de l'ajout des documents : {str(e)}")

Documents ajoutés avec IDs : ['doc_-2412211889263932192', 'doc_-4052915018708320956']


In [7]:
question = "Qu'est-ce qu'un modèle Transformer ?"

try:
    # Récupération et construction du prompt
    prompt_final = retriever.recuperer_et_construire(
        question=question,
        seuil_similarite=0.65
    )
    
    print("\n=== Prompt généré ===")
    print(prompt_final)
    
    # Affichage des métriques
    print("\n=== Métriques ===")
    print(f"Latence totale : {retriever.last_latency:.2f}s")
    print(f"Coût estimé : €{retriever.last_metrics.cout_euros:.5f}")
    print(f"Tokens d'entrée : {retriever.last_metrics.tokens_entree}")
    
except Exception as e:
    print(f"Erreur lors du traitement : {str(e)}")


Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2
Aucun document ne dépasse le seuil de similarité.



=== Prompt généré ===

Contexte:
Aucun document pertinent trouvé.

Question: Qu'est-ce qu'un modèle Transformer ?

Instructions:
- Utilisez uniquement les informations fournies dans le contexte ci-dessus
- Si vous ne pouvez pas répondre à partir du contexte, dites-le clairement
- Citez les passages pertinents du contexte pour justifier votre réponse
- Restez factuel et précis

Réponse:

=== Métriques ===
Latence totale : 0.27s
Coût estimé : €0.00000
Tokens d'entrée : 0


In [8]:
def charger_et_indexer_pdf(chemin_pdf: str):
    try:
        # Lecture et découpage du PDF
        texte_complet = vector_store.lire_pdf(chemin_pdf)
        chunks = vector_store.decouper_texte(texte_complet, taille_chunk=300)
        
        # Indexation des chunks
        retriever.ajouter_documents(
            textes=chunks,
            metadonnees=[{"source": chemin_pdf}] * len(chunks)
        )
        print(f"\nPDF '{chemin_pdf}' indexé avec {len(chunks)} chunks")
        
    except Exception as e:
        print(f"Erreur PDF : {str(e)}")

charger_et_indexer_pdf("sample.pdf")


PDF 'sample.pdf' indexé avec 1 chunks
